## 네이버 뉴스 본문 수집 함수 생성

- 네이버뉴스 링크의 도메인에 따라 본문 수집 코드를 다르게 작성해야 함
  - `https://n.news`로 시작하는 뉴스 본문은 nlink로 HTTP 요청 실행
  - `https://m.sports`로 시작하는 뉴스 본문은 `https://api-gw.sports`로 변경해야 함
  - `https://m.entertain`으로 시작하는 뉴스 본문은 `https://api-gw.entertain`으로 변경해야 함

In [1]:
# 라이브러리 호출
import os
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bts
import json
import re
import time
from tqdm.notebook import tqdm

In [2]:
# 현재 작업 경로 확인
os.getcwd()

'/Users/seonghona/Documents/Lectures/Customer/DBR/code'

In [3]:
# data 폴더로 작업 경로 변경
os.chdir(path = '../data')

In [4]:
# 현재 작업 경로에 있는 폴더명과 파일명 확인
os.listdir()

['Naver_News_List.pkl']

In [5]:
# pkl 파일을 읽고 newsList 생성
newsList = pd.read_pickle(filepath_or_buffer = 'Naver_News_List.pkl')

In [6]:
# newsList의 처음 5행 확인
newsList.head()

,press,title,nlink
0,뉴시스,"올림픽 축구 탈락했는데 파리간 정몽규…""냄비받침 선물하러 갔나""",https://n.news.naver.com/mnews/article/003/001...
1,서울신문,"올림픽 축구 탈락했는데…정몽규, 파리서 FIFA 회장 만났다",https://n.news.naver.com/mnews/article/081/000...
2,SBS Biz,'국민욕받이'라는 정몽규 회장…의문의 1패 [CEO 업앤다운],https://n.news.naver.com/mnews/article/374/000...
3,조선비즈,“축구협회장도 맡아주세요” 정의선 비교에 씁쓸한 정몽규,https://n.news.naver.com/mnews/article/366/000...
4,노컷뉴스,"정몽규는 헛발질, 정의선은 백발백중[파리올림픽]",https://n.news.naver.com/mnews/article/079/000...


In [7]:
# newsList의 정보 확인
newsList.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   press   77 non-null     object
 1   title   77 non-null     object
 2   nlink   77 non-null     object
dtypes: object(3)
memory usage: 1.9+ KB


In [8]:
# 네이버뉴스 링크로 HTTP 요청을 실행하고 뉴스 본문을 반환하는 함수 생성
def NaverNewsBody(nlink):
    
    # nlink에 따라 HTTP 요청 실행 및 응답 바디 문자열 처리 코드 분기
    if 'https://n.news' in nlink:
        
        # 네이버뉴스 링크로 HTTP 요청 실행
        res = requests.get(url = nlink)
        
        # HTTP 응답 바디 문자열을 bs4.BeautifulSoup 객체로 변환
        soup = bts(markup = res.text, features = 'html.parser')
        
        # soup에서 뉴스 본문을 포함하는 HTML 요소 선택
        item = soup.select_one(selector = '#dic_area, div._article_content')
        
        # item의 텍스트 반환
        # [참고] strip()은 문자열 양 옆에 있는 공백을 모두 제거함
        return item.text.strip()
    
    elif 'https://m' in nlink:
        
        # 'https://m'으로 시작하는 nlink의 일부 문자열 변경한 url 생성
        url = re.sub(pattern = 'https://m.(sports|entertain).naver.com(.+)?(?=/article)', 
                     repl = r'https://api-gw.\1.naver.com/news', 
                     string = newsList['nlink'].iloc[i])
        
        # url로 HTTP 요청 실행
        res = requests.get(url = url)
        
        # JSON 형태의 문자열을 딕셔너리로 변환
        dic = json.loads(s = res.text)
        
        # 뉴스 본문 반환
        return dic['result']['articleInfo']['article']['content']

In [9]:
# 인덱스 설정: 첫 번째 원소
# [참고] 반복문으로 실행할 코드에 인덱스를 추가하면 편리함
i = 0

In [10]:
# 해당 인덱스의 nlink 확인
print(newsList['nlink'].iloc[i])

https://n.news.naver.com/mnews/article/003/0012717267


In [11]:
# 함수 테스트
NaverNewsBody(nlink = newsList['nlink'].iloc[i])

'인판티노 FIFA 회장, SNS에 사진 공개정몽규 회장, FIFA 회장에 친필서명 담긴 자서전 선물\n\n\n\n[서울=뉴시스] 잔니 인판티노 국제축구연맹(FIFA) 회장이 7일(한국시각) 자신의 사회관계망서비스(SNS)에 정몽규 대한축구협회 회장과 만난 사진을 게시했다. (사진=잔니 인판티노 FIFA 회장 SNS 캡처) 2024.08.07. *재판매 및 DB 금지[서울=뉴시스] 최윤서 인턴 기자 = 대한민국 축구 대표팀이 충격의 예선 탈락으로 40년 만에 올림픽 출전이 좌절된 가운데, 파리올림픽이 열리고 있는 프랑스 파리를 방문해 잔니 인판티노 국제축구연맹(FIFA·피파) 회장을 만난 정몽규 대한축구협회 회장의 행보가 누리꾼들의 빈축을 사고 있다.두 사람의 만남은 7일(한국시각) 인판티노 회장이 자신의 사회관계망서비스(SNS)에 정몽규 회장과 함께 찍은 여러 장의 사진을 게시하면서 알려졌다.인판티노 회장은 SNS 게시글에 "나의 오랜 친구인 정몽규 회장을 프랑스 파리의 FIFA 사무소에서 만나게 돼 매우 기쁘다"며 반가운 마음을 드러냈다.그러면서 인판티노 회장은 "정 회장이 자신의 친필 서명이 담긴 자서전 \'축구의 시대\' 한 권을 선물해 줬다. 감사하다. 직접 쓴 글을 읽게 돼 영광"이라며 "정 회장에 대해, 그리고 그의 경험에 대해 더 많이 알 수 있길 기대한다"고 말했다.정 회장이 직접 서명한 자서전을 같이 들고 미소 지으며 촬영한 기념사진도 함께 올렸다.이 같은 소식이 알려지자 국내 축구 팬들과 축구계는 강하게 반발했다. \n\n\n\n정몽규 대한축구협회장(왼쪽)이 잔니 인판티노 국제축구연맹(FIFA‧피파) 회장에게 자서전을 선물한 뒤 같이 읽어보고 있다. (사진=FIFA 제공) *재판매 및 DB 금지대한축구협회는 위르겐 클린스만 전 감독 경질 이후 U-23 대표팀을 이끌고 있던 황선홍 감독을 A대표팀 사령탑을 겸직하게 하며 2026 북중미 월드컵 지역 2차 예선을 치르게 했다.황 감독은 파리 올림픽 예선을 겸한 U-23 아시안컵 8강전에서 신태용

In [12]:
# 경고를 출력하지 않도록 설정
import warnings
warnings.filterwarnings('ignore')

In [13]:
# 반복 실행할 횟수 생성
n = len(newsList)
print(n)

77


In [14]:
# newsList에 뉴스 본문을 저장할 열 추가
newsList['body'] = np.nan

# 반복문으로 네이버뉴스 본문 수집
for i in tqdm(range(n)):
    
    # 네이버뉴스 링크로 뉴스 본문 수집
    newsList['body'].iloc[i] = NaverNewsBody(nlink = newsList['nlink'].iloc[i])
    
    # 1초간 멈춤
    time.sleep(1)

  0%|          | 0/77 [00:00<?, ?it/s]

In [15]:
# newsList의 처음 5행 확인
newsList.head()

,press,title,nlink,body
0,뉴시스,"올림픽 축구 탈락했는데 파리간 정몽규…""냄비받침 선물하러 갔나""",https://n.news.naver.com/mnews/article/003/001...,"인판티노 FIFA 회장, SNS에 사진 공개정몽규 회장, FIFA 회장에 친필서명 ..."
1,서울신문,"올림픽 축구 탈락했는데…정몽규, 파리서 FIFA 회장 만났다",https://n.news.naver.com/mnews/article/081/000...,"인판티노 FIFA 회장, SNS에 사진 공개“친구 만나 반가워”…정 회장, 에세이 ..."
2,SBS Biz,'국민욕받이'라는 정몽규 회장…의문의 1패 [CEO 업앤다운],https://n.news.naver.com/mnews/article/374/000...,"정몽규 HDC그룹 회장, 요즘 난감한 상황이 이어지고 있습니다. 당장 5촌 조카 정..."
3,조선비즈,“축구협회장도 맡아주세요” 정의선 비교에 씁쓸한 정몽규,https://n.news.naver.com/mnews/article/366/000...,"40년 만에 올림픽 출전 불발, 깜깜이 감독 선발 등으로 논란을 겪고 있는 정몽규 ..."
4,노컷뉴스,"정몽규는 헛발질, 정의선은 백발백중[파리올림픽]",https://n.news.naver.com/mnews/article/079/000...,정몽규 대한축구협회장이 2월 16일 오후 서울 종로구 축구회관에서 브리핑을 갖고 클...


In [16]:
# body에서 HTML 요소 문자열 삭제
newsList['body'] = newsList['body'].str.replace(pat = '<.+?>', repl = '', regex = True)

In [17]:
# body의 처음 5개 확인
newsList['body'].head()

0    인판티노 FIFA 회장, SNS에 사진 공개정몽규 회장, FIFA 회장에 친필서명 ...
1    인판티노 FIFA 회장, SNS에 사진 공개“친구 만나 반가워”…정 회장, 에세이 ...
2    정몽규 HDC그룹 회장, 요즘 난감한 상황이 이어지고 있습니다. 당장 5촌 조카 정...
3    40년 만에 올림픽 출전 불발, 깜깜이 감독 선발 등으로 논란을 겪고 있는 정몽규 ...
4    정몽규 대한축구협회장이 2월 16일 오후 서울 종로구 축구회관에서 브리핑을 갖고 클...
Name: body, dtype: object

In [18]:
# 현재 작업 경로 확인
os.getcwd()

'/Users/seonghona/Documents/Lectures/Customer/DBR/data'

In [19]:
# newsList를 pkl 파일로 저장
pd.to_pickle(obj = newsList, filepath_or_buffer = 'Naver_News_List.pkl')

In [20]:
# 현재 작업 경로에 있는 폴더명과 파일명 확인
os.listdir()

['Naver_News_List.pkl']

## End of Document